In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

# Set max rows and columns to display
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)



In [10]:

# Configure logging
from logger_script import SetupLogger
 

logger = SetupLogger(log_file='../logs/modeling_notebooks.log').get_logger()

In [20]:
from load_data import LoadData  
data = LoadData('../data/processed__data.csv', logger=logger)
test = LoadData('../data/cs-test.csv', logger=logger)
test_df = test.load_dataset()
data_df = data.load_dataset()
data_df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [29]:
from data_preparation import DataPreparation
# Assuming df_creditcard is the DataFrame for the credit card dataset
_data_df = DataPreparation(data_df, target_column='SeriousDlqin2yrs')
_data_df.train_test_split(test_size=0.2, random_state=42)

# Retrieving the train and test sets
X_train, X_test, y_train, y_test= _data_df.get_train_test_data()

Data split into training and testing sets successfully.


In [31]:
# Create instance of the pipeline
model_pipeline = ModelPipeline(X_train, X_test, y_train ,y_test)

# add models
# model_pipeline.add_models()
# Train and evaluate, logging with MLflow
best_model, best_model_name = model_pipeline.train_and_evaluate()
# Save the best model
model_pipeline.save_best_models(best_model, best_model_name, 'fraud')

Tuning hyperparameters for Random Forest...
Random Forest best parameters: {'classifier__max_depth': 10, 'classifier__n_estimators': 50}
Tuning hyperparameters for Gradient Boosting...
Gradient Boosting best parameters: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 50}
Random Forest took 4.05 seconds to train


2025/02/23 16:58:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'random_forest' already exists. Creating a new version of this model...
Created version '2' of model 'random_forest'.


Random Forest model trained and logged with MLflow
Gradient Boosting took 14.67 seconds to train


2025/02/23 16:58:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'gradient_boosting' already exists. Creating a new version of this model...
Created version '2' of model 'gradient_boosting'.


Gradient Boosting model trained and logged with MLflow
752/752 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


...

2025/02/23 16:59:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'lstm' already exists. Creating a new version of this model...
Created version '2' of model 'lstm'.


LSTM model trained and logged with MLflow
752/752 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step
CNN took 23.29 seconds to train


2025/02/23 17:00:07 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/02/23 17:00:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


CNN model trained and logged with MLflow
Random Forest best model saved.


Registered model 'cnn' already exists. Creating a new version of this model...
Created version '2' of model 'cnn'.


In [32]:
# Get the results 
results_fraud, y_probs_fraud = model_pipeline.get_results()

In [33]:
pd.DataFrame(results_fraud).T

,Accuracy,Precision,Recall,F1 Score,ROC AUC
Random Forest,0.932984,0.600000,0.151943,0.242481,0.852944
Gradient Boosting,0.932943,0.587992,0.167256,0.260431,0.852271
LSTM,0.932943,0.619048,0.130153,0.215085,0.851164
CNN,0.929201,0.333333,0.002945,0.005838,0.601443


 ### Model Explainability

Model explainability is crucial for understanding, trust, and debugging in machine learning models. we will use SHAP (SHapley Additive exPlanations) and LIME (Local Interpretable Model-agnostic Explanations) to interpret the models I built for fraud detection.



In [40]:
# # Import neccessary modules
from model_explainer import ModelExplainer

# Create instance of the class
fraud_explainer = ModelExplainer('../app/gradient_boosting_fraud_best_model.pkl', X_test)

 # Explains the first instance in the dataset
fraud_explainer.explain_model(instance_idx=0) 

ImportError: cannot import name 'ModelExplainer' from 'model_explainer' (/home/abel/CODE/AAiT/AI/Advanced-XAI-Project-Credit-Score-Modeling/scripts/model_explainer.py)